<h3>Import The Required Libraries</h3>

In [ ]:
import praw
import pandas as pd
import nltk
import re
import datetime as dt
#nltk.download('stopwords')
from nltk.corpus import stopwords

<h3> variable declaration </h3>

In [ ]:
flairs = ["AskIndia", "Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food", "AMA"]
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_?]')
#STOPWORDS = set(stopwords.words('english'))

<h3> utility functions </h3><br>
clean_text function will remove unneccesary symbols

In [ ]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

def string_form(value):
    return str(value)

def clean_text(text):
   
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    #text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

<h3> scraping reddit data </h3><br>
There are total 12 fliars. I collected only 100 posts per fliar as I think they should be sufficient for a POC, and due to lack of computational resources.


In [ ]:
reddit = praw.Reddit(client_id = "0hliVZBlDhlipQ",client_secret = "pjUJDMMXhYaEmyJaLkQYb_2Fccg",user_agent = "Reddit Flair Detector",username = "chandan21121998",password = "Chandan@1234")
subreddit = reddit.subreddit('india')
topics_dict = {"flair":[], "title":[], "score":[], "id":[], "url":[], "comms_num": [], "created": [], "body":[], "author":[], "comments":[]}

for flair in flairs:
    print(flair)
    get_subreddits = subreddit.search(flair, limit=100)
    #print('len', len(list(get_subreddits)))
    for submission in get_subreddits:
        topics_dict["flair"].append(flair)
        topics_dict["title"].append(submission.title)
        topics_dict["score"].append(submission.score)
        topics_dict["id"].append(submission.id)
        topics_dict["url"].append(submission.url)
        topics_dict["comms_num"].append(submission.num_comments)
        topics_dict["created"].append(submission.created)
        topics_dict["body"].append(submission.selftext)
        topics_dict["author"].append(submission.author)
    
        submission.comments.replace_more(limit=None)
        comment = ''
        comment_c = 0
        for top_level_comment in submission.comments:
            comment = comment + ' ' + top_level_comment.body
            comment_c += 1
            if comment_c> 20:
                break
        topics_dict["comments"].append(comment)
topics_data = pd.DataFrame(topics_dict)



<h3> preprocess and save the data </h3>

In [ ]:
_timestamp = topics_data["created"].apply(get_date)
topics_data = topics_data.assign(timestamp = _timestamp)
topics_data.to_csv('reddit-india-data.csv', index=False) 

In [ ]:
topics_data = pd.read_csv('reddit-india-data.csv')
topics_data.head()

In [ ]:

topics_data['title'] = topics_data['title'].apply(string_form)
topics_data['body'] = topics_data['body'].apply(string_form)
topics_data['comments'] = topics_data['comments'].apply(string_form)

topics_data['title'] = topics_data['title'].apply(clean_text)
topics_data['body'] = topics_data['body'].apply(clean_text)
topics_data['comments'] = topics_data['comments'].apply(clean_text)
topics_data.to_csv('reddit-india-data.csv', index=False) 